<a href="https://colab.research.google.com/github/GochiStuff/mnist/blob/main/MNIST_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

So I will try to build a number identify Netwowrk , and see if everything will go fine !

Starting off I will get the required data !

In [3]:
import numpy as np

In [4]:
from tensorflow.keras.datasets import mnist

# Load data
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = mnist.load_data()

# Flatten 28x28 images to 784-dim vectors
X_train = X_train_raw.reshape(-1, 784).astype(np.float32) / 255.0  # (60000, 784)
X_test  = X_test_raw.reshape(-1, 784).astype(np.float32) / 255.0   # (10000, 784)


Let me write what I am going to do ! :

1. Z = WX + b
  here W -> shape

In [5]:
# One-hot encode labels
def one_hot(y, num_classes=10):
    return np.eye(num_classes)[y]

Y_train = one_hot(y_train_raw)  # (60000, 10)
Y_test  = one_hot(y_test_raw)   # (10000, 10)

print(X_train[2])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [6]:
# print(X.shape)  # (60000, 784)
# print(Y.shape)  # (60000, 10)
# print(Y[:20])

In [7]:
# starting off with making helper functions

In [8]:

# -----------------------------
# Activation Functions
# -----------------------------

def sigmoid(z):
    """Sigmoid activation function."""
    return 1 / (1 + np.exp(-z))

def sigmoid_derivative(z):
    """Derivative of the sigmoid function."""
    s = sigmoid(z)
    return s * (1 - s)

def softmax(z):
    """Softmax activation function (for output layer)."""
    exp_z = np.exp(z - np.max(z, axis=1, keepdims=True))  # stability
    return exp_z / np.sum(exp_z, axis=1, keepdims=True)

def ReLU(x):
    """ReLU activation function."""
    return np.maximum(0, x)


def ReLU_derivative(x):
    return (x > 0).astype(float)
# -----------------------------
# Loss Function
# -----------------------------

def cross_entropy_loss(y_true, y_pred):
    """Cross-entropy loss for multiclass classification."""
    m = y_true.shape[0]
    epsilon = 1e-12  # prevent log(0)
    y_pred = np.clip(y_pred, epsilon, 1. - epsilon)
    loss = -np.sum(y_true * np.log(y_pred)) / m
    return loss

Lets start making the forward popagation .

important parameters declaration .

In [9]:
# W = np.zeros((784, 10))
# b = np.zeros(10)

In [10]:
# def calculate_z( W , X , b) :
#   Z = np.dot(X , W) + b
#   return Z

In [11]:
print(calculate_z(W , X , b)[:20])

NameError: name 'calculate_z' is not defined

time to calculate A

In [12]:
# rather than parts let me just build the complete model now .

In [23]:
class Model:
  def __init__(self, X_train, Y_train, epochs=300, learning_rate=0.1, batch_size=128):

    self.W1 = np.random.randn(784, 10) * np.sqrt(2. / 784)  # Input to hidden
    self.W2 = np.random.randn(10, 10) * np.sqrt(2. / 10)  # Hidden to output
    self.b1 = np.zeros(10)
    self.b2 = np.zeros(10)
    self.batch_size = batch_size
    self.epochs = epochs
    self.lr = learning_rate
    self.X_train = X_train
    self.Y_train = Y_train
    self.m = X_train.shape[0]


  def forward_propagation(self, X):
      Z1 = np.dot(X, self.W1) + self.b1
      A1 = ReLU(Z1)

      Z2 = np.dot(A1, self.W2) + self.b2
      A2 = softmax(Z2)

      return Z1, A1, Z2, A2


  def runModel( self ):

    initial_lr = self.lr
    for epoch in range(self.epochs):
      # shuffle at each epoch
      indices = np.random.permutation(self.m)
      X_shuffled = self.X_train[indices]
      Y_shuffled = self.Y_train[indices]

      for i in range ( 0 , self.m , self.batch_size):
        X_batch = X_shuffled[i:i+self.batch_size]
        Y_batch = Y_shuffled[i:i+self.batch_size]

        Z1, A1, Z2, A2 = self.forward_propagation(X_batch)

        # Now backprop works:
        dZ2 = A2 - Y_batch
        dW2 = np.dot(A1.T, dZ2) / self.batch_size
        db2 = np.sum(dZ2, axis=0) / self.batch_size

        dA1 = np.dot(dZ2, self.W2.T)
        dZ1 = dA1 * ReLU_derivative(Z1)
        dW1 = np.dot(X_batch.T, dZ1) / self.batch_size
        db1 = np.sum(dZ1, axis=0) / self.batch_size


        # Update parameters
        self.W2 -= self.lr * dW2
        self.b2 -= self.lr * db2
        self.W1 -= self.lr * dW1
        self.b1 -= self.lr * db1



      _, _, _, A_all = self.forward_propagation(self.X_train)
      loss = cross_entropy_loss(self.Y_train, A_all)
      acc = np.mean(np.argmax(A_all, axis=1) == np.argmax(self.Y_train, axis=1))
      if epoch % 5 == 0:
          print(f"Epoch {epoch}, Loss: {loss:.4f}, Accuracy: {acc*100:.2f}%")
      self.lr = initial_lr * (0.98 ** (epoch // 10))



In [24]:
# trying model out !

mnist_model = Model(X_train, Y_train)
mnist_model.runModel()


Epoch 0, Loss: 0.3583, Accuracy: 89.89%
Epoch 5, Loss: 0.2450, Accuracy: 93.02%
Epoch 10, Loss: 0.2142, Accuracy: 93.97%
Epoch 15, Loss: 0.2039, Accuracy: 94.29%
Epoch 20, Loss: 0.1899, Accuracy: 94.58%
Epoch 25, Loss: 0.1874, Accuracy: 94.64%
Epoch 30, Loss: 0.1811, Accuracy: 94.84%
Epoch 35, Loss: 0.1762, Accuracy: 94.89%
Epoch 40, Loss: 0.1685, Accuracy: 95.14%
Epoch 45, Loss: 0.1718, Accuracy: 94.91%
Epoch 50, Loss: 0.1745, Accuracy: 94.86%
Epoch 55, Loss: 0.1648, Accuracy: 95.18%
Epoch 60, Loss: 0.1681, Accuracy: 94.94%
Epoch 65, Loss: 0.1575, Accuracy: 95.36%
Epoch 70, Loss: 0.1598, Accuracy: 95.23%
Epoch 75, Loss: 0.1552, Accuracy: 95.46%
Epoch 80, Loss: 0.1547, Accuracy: 95.47%
Epoch 85, Loss: 0.1514, Accuracy: 95.54%
Epoch 90, Loss: 0.1504, Accuracy: 95.59%
Epoch 95, Loss: 0.1629, Accuracy: 95.07%
Epoch 100, Loss: 0.1473, Accuracy: 95.71%
Epoch 105, Loss: 0.1486, Accuracy: 95.62%
Epoch 110, Loss: 0.1522, Accuracy: 95.52%
Epoch 115, Loss: 0.1475, Accuracy: 95.68%
Epoch 120, Los